## Reconstruct MNIST with AutoEncoder 


### Import Packages

In [97]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms,datasets

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

print('pytorch version:',torch.__version__,
      '\ntorchvision version: ',torchvision.__version__,
      '\nnumpy version:' ,np.__version__,
      '\nmatplotlib version: ',matplotlib.__version__)

pytorch version: 1.5.0 
torchvision version:  0.6.0.dev20200319 
numpy version: 1.18.2 
matplotlib version:  3.2.1


### Settings

In [98]:
# model runs on  GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 1e-3 
num_epochs = 10
batch_size = 64

### Dataset: MNIST


In [99]:
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.Compose([
                                    transforms.ToTensor(),
                                ]),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.Compose([
                                    transforms.ToTensor(),
                                ]))

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

### Define AE

In [100]:
class AE(nn.Module):

    def __init__(self):
        super(AE, self).__init__()
        # [b, 784] => [b, 20]
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU()
        )
        # [b, 20] => [b, 784]
        self.decoder = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), 784)
        # encoder
        x = self.encoder(x)
        # decoder
        x = self.decoder(x)
        # reshape
        x = x.view(x.size(0), 1, 28, 28)

        return x
    

### Init AE, define optimizer and loss function

In [101]:
model = AE()
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criteon = nn.MSELoss().to(device)

### Training AE

In [ ]:
# %matplotlib inline

result = []

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (x, _) in enumerate(train_loader):
        x = x.to(device)
        
        # forward
        x_hat = model(x) # x_hat is the image reconstructed by the model
        loss = criteon(x_hat, x)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
    
    model.eval()
    with torch.no_grad():
        for x, _ in test_loader:
            x = x.to(device)
            x_hat = model(x)
            
            result.append(x_hat)
    
# visual reconstruction image
x_hat = result[-1].view(batch_size, 28, 28).cpu()
plt.figure()
for i in range(1, batch_size):
    plt.subplot(8, int(batch_size / 8), i)
    plt.imshow(x_hat[i-1], cmap=plt.cm.gray)
    # plt.xticks([])
    # plt.yticks([])
plt.show()

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.232859
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.232527
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.232240
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.232683
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.233295
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.231830
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.231300
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.232265
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.232676
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.231861
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.231788
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.231614
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.232248
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.232123
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.232265
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.232071
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.231945
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.232282
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.232442
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.231383
Train Epoch: